# Identifying Nanoparticles Aggregate from Its Scattering Spectra with Machine Learning

In [1]:
from pandas import read_excel
from pandas.plotting import scatter_matrix
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
import numpy as np

In [43]:
# Load dataset
url = "../../data/processed/den/tanpabola3.xlsx"
names = ['lam_max1', 'csc_max1', 'lam_min', 'csc_min','lam_fwhm1','c_mid','lam_max2','csc_max2','lam_fwhm2','fwhm','posisi1','posisi2','posisi3','posisi4','posisi5','arah_k','arah_E','sb_putar','sudut1','sudut2','ket'] 
dataset = read_excel(url, names=names, header=None)

print(dataset.head(5))

   lam_max1      csc_max1  lam_min       csc_min  lam_fwhm1         c_mid  \
0       408  2.774638e-14      434  2.264798e-14        464  4.239645e-14   
1       410  2.984472e-14      432  2.725996e-14        464  4.555423e-14   
2       362  2.400898e-14      390  1.436011e-14        442  4.153600e-14   
3       364  2.971955e-14      392  1.702803e-14        412  4.696978e-14   
4       366  3.552294e-14      392  1.962838e-14        412  5.500586e-14   

   lam_max2      csc_max2  lam_fwhm2  fwhm  ...  posisi2  posisi3  posisi4  \
0       474  6.214492e-14        484    20  ...        0        0        0   
1       474  6.384849e-14        484    20  ...        0        0        0   
2       472  6.871189e-14        488    46  ...        0        0        0   
3       470  7.691152e-14        488    76  ...        0        0        0   
4       462  9.038334e-14        488    76  ...        0        0        0   

   posisi5  arah_k  arah_E  sb_putar  sudut1  sudut2  ket  
0       

In [44]:
# Define input and output data
X = dataset.iloc[:,0:10].values
y = dataset.iloc[:,20].values

In [4]:
# Spot check algorithms
models = {
    'LR' : make_pipeline(StandardScaler(), LogisticRegression(solver='liblinear', multi_class='ovr')),
    'LDA' : LinearDiscriminantAnalysis(),
    'KNN' : KNeighborsClassifier(),
    'CART' : DecisionTreeClassifier(),
    'NB' : GaussianNB(),
    'SVM' : SVC(gamma='auto')
}

In [45]:
# Spot check algorithms
models = {
    'LR' : make_pipeline(StandardScaler(), LogisticRegression(solver='liblinear', multi_class='ovr')),
    'LDA' : LinearDiscriminantAnalysis(),
    'KNN' : KNeighborsClassifier(),
    'CART' : DecisionTreeClassifier(),
    'NB' : GaussianNB(),
    'SVM' : make_pipeline(StandardScaler(), SVC(gamma='auto'))
}

In [46]:
#Cross-validation
kf = StratifiedKFold(n_splits=2, shuffle=True, random_state=42)

# Evaluation storage
results = {name: {'accuracy': [], 'precision': []} for name in models}

# Training and evaluation
for name, model in models.items():
    for train_idx, test_idx in kf.split(X, y):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        acc = accuracy_score(y_test, y_pred)
        prec = precision_score(y_test, y_pred, average=None)

        results[name]['accuracy'].append(acc)
        results[name]['precision'].append(prec)

# Print results
for name, scores in results.items():
    print(f"Model: {name}")
    print(f"  Mean Accuracy: {np.mean(scores['accuracy']):.4f}")
    print(f"  Mean Precision: {np.mean(scores['precision']):.4f}")
    print()


Model: LR
  Mean Accuracy: 0.7834
  Mean Precision: 0.7795

Model: LDA
  Mean Accuracy: 0.7433
  Mean Precision: 0.7482

Model: KNN
  Mean Accuracy: 0.8983
  Mean Precision: 0.9007

Model: CART
  Mean Accuracy: 0.9151
  Mean Precision: 0.9323

Model: NB
  Mean Accuracy: 0.5723
  Mean Precision: 0.5542

Model: SVM
  Mean Accuracy: 0.9085
  Mean Precision: 0.9165



## 1. Validation for `tanpabola3.xlsx` Data

In [20]:
# Load dataset
url = "../../data/processed/den/tanpabola3.xlsx"
names = ['lam_max1', 'csc_max1', 'lam_min', 'csc_min','lam_fwhm1','c_mid','lam_max2','csc_max2','lam_fwhm2','fwhm','posisi1','posisi2','posisi3','posisi4','posisi5','arah_k','arah_E','sb_putar','sudut1','sudut2','ket'] 
dataset = read_excel(url, names=names, header=None)

In [21]:
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=1,
    random_state=1
)

print(len(y_train))
print(len(y_valid))
print(len(y_valid)/(len(y_train) + len(y_valid)))

1070
1
0.0009337068160597573


In [22]:
# Write results to a text file
with open("output_validation_tanpabola3.txt", "w") as f:
    for name, model in models.items():
        model.fit(X_train,y_train)
        y_predict = model.predict(X_valid)
        report = classification_report(y_valid, y_predict)
        cf = confusion_matrix(y_valid, y_predict)

        f.write(f"Model: {name}\n")
        f.write(f"confusion matrix:\n {cf}\n")
        f.write(f"classification report: \n {report}\n")
        f.write(f"=====================================================\n")
print("Evaluation results exported to .txt file") 


d:\1. Project\Python\mlvenv\Lib\site-packages\sklearn\metrics\_classification.py:386: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(
d:\1. Project\Python\mlvenv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\1. Project\Python\mlvenv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\1. Project\Python\mlvenv\Lib\site-packages\sklearn\metrics\_classification.py:1509: Und

Evaluation results exported to .txt file


d:\1. Project\Python\mlvenv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\1. Project\Python\mlvenv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\1. Project\Python\mlvenv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\1. Project\Python\mlvenv\Lib\site-pa

### - Testing with Data `3bola.xlsx`

In [23]:
# Load dataset
url = "../../data/processed/den/3bola.xlsx"
names = ['lam_max1', 'csc_max1', 'lam_min', 'csc_min','lam_fwhm1','c_mid','lam_max2','csc_max2','lam_fwhm2','fwhm','posisi1','posisi2','posisi3','posisi4','posisi5','arah_k','arah_E','sb_putar','sudut1','sudut2','ket'] 
bola3 = read_excel(url, names=names, header=None)

# Define input and output data
X_bola3 = bola3.iloc[:,0:10].values
y_bola3 = bola3.iloc[:,20].values

print(len(X_bola3))
print(len(y_bola3))

630
630


In [24]:
y_valid = y_bola3
X_valid = X_bola3
# Write results to a text file
with open("output_validation_bola3_100.txt", "w") as f:
    for name, model in models.items():
        model.fit(X_train,y_train)
        y_predict = model.predict(X_valid)
        report = classification_report(y_valid, y_predict)
        cf = confusion_matrix(y_valid, y_predict)

        f.write(f"Model: {name}\n")
        f.write(f"confusion matrix:\n {cf}\n")
        f.write(f"classification report: \n {report}\n")
        f.write(f"=====================================================\n")
print("Evaluation results exported to .txt file")  

Evaluation results exported to .txt file


### - Testing with 20 data `bola3.xlsx` on each group 

In [40]:
y_valid = y_bola3
X_valid = X_bola3
# Write results to a text file
with open("output_validation_bola3_20-50.txt", "w") as f:
    for i in range(int(np.round(len(y_bola3)/50))):
        row = i*50
        itv = 20 + (i*50)
        X_valid = X_bola3[row:itv,:] 
        y_valid = y_bola3[row:itv]

        models['SVM'].fit(X_train,y_train)
        y_predict1 = models['SVM'].predict(X_valid)
        report1 = classification_report(y_valid, y_predict1)
        cf1 = confusion_matrix(y_valid, y_predict1)

        models['CART'].fit(X_train,y_train)
        y_predict2 = models['CART'].predict(X_valid)
        report2 = classification_report(y_valid, y_predict2)
        cf2 = confusion_matrix(y_valid, y_predict2)

        f.write(f"=========================( {i+1} )==========================\n")
        f.write(f"Model: SVM\n")
        f.write(f"confusion matrix:\n {cf1}\n")
        f.write(f"classification report: \n {report1}\n")
        f.write(f"--------------------------------------\n")
        f.write(f"Model: CART\n")
        f.write(f"confusion matrix:\n {cf2}\n")
        f.write(f"classification report: \n {report2}\n")
print("Evaluation results exported to .txt file")  

d:\1. Project\Python\mlvenv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\1. Project\Python\mlvenv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\1. Project\Python\mlvenv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\1. Project\Python\mlvenv\Lib\site-packages\sklearn\m

Evaluation results exported to .txt file


d:\1. Project\Python\mlvenv\Lib\site-packages\sklearn\metrics\_classification.py:386: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(
d:\1. Project\Python\mlvenv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\1. Project\Python\mlvenv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\1. Project\Python\mlvenv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMe

## 2. Validation for `tanpabola4.xlsx` Data

In [ ]:
# Load dataset
url = "../../data/processed/den/tanpabola4.xlsx"
names = ['lam_max1', 'csc_max1', 'lam_min', 'csc_min','lam_fwhm1','c_mid','lam_max2','csc_max2','lam_fwhm2','fwhm','posisi1','posisi2','posisi3','posisi4','posisi5','arah_k','arah_E','sb_putar','sudut1','sudut2','ket'] 
dataset = read_excel(url, names=names, header=None)

# Define input and output data
X = dataset.iloc[:,0:10].values
y = dataset.iloc[:,20].values


In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.50,
    random_state=2
)

print(len(y_train))
print(len(y_valid))
print(len(y_valid)/(len(y_train) + len(y_valid)))

In [ ]:
# Write results to a text file
with open("output_validation_tanpabola4.txt", "w") as f:
    for name, model in models.items():
        model.fit(X_train,y_train)
        y_predict = model.predict(X_valid)
        report = classification_report(y_valid, y_predict)
        cf = confusion_matrix(y_valid, y_predict)

        f.write(f"Model: {name}\n")
        f.write(f"confusion matrix:\n {cf}\n")
        f.write(f"classification report: \n {report}\n")
        f.write(f"=====================================================\n")
print("Evaluation results exported to .txt file")  

## 3. Validation for `bola12345.xlsx` Data

In [ ]:
# Load dataset
url = "../../data/processed/den/bola12345.xlsx"
names = ['lam_max1', 'csc_max1', 'lam_min', 'csc_min','lam_fwhm1','c_mid','lam_max2','csc_max2','lam_fwhm2','fwhm','posisi1','posisi2','posisi3','posisi4','posisi5','arah_k','arah_E','sb_putar','sudut1','sudut2','ket'] 
dataset = read_excel(url, names=names, header=None)

# Define input and output data
X = dataset.iloc[:,0:10].values
y = dataset.iloc[:,20].values

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=1,
    random_state=2
)

print(len(y_train))
print(len(y_valid))
print(len(y_valid)/(len(y_train) + len(y_valid)))

In [ ]:
# Write results to a text file
with open("output_validation_bola12345.txt", "w") as f:
    for name, model in models.items():
        model.fit(X_train,y_train)
        y_predict = model.predict(X_valid)
        report = classification_report(y_valid, y_predict)
        cf = confusion_matrix(y_valid, y_predict)

        f.write(f"Model: {name}\n")
        f.write(f"confusion matrix:\n {cf}\n")
        f.write(f"classification report: \n {report}\n")
        f.write(f"=====================================================\n")
print("Evaluation results exported to .txt file")

### - Testing Experiment Data

In [ ]:
# Load dataset
url = "../../data/processed/den/Exp_2bola.xlsx"
names = ['lam_max1', 'csc_max1', 'lam_min', 'csc_min','lam_fwhm1','c_mid','lam_max2','csc_max2','lam_fwhm2','fwhm','posisi1','posisi2','posisi3','posisi4','posisi5','arah_k','arah_E','sb_putar','sudut1','sudut2','ket'] 
exp = read_excel(url, names=names, header=None)

# Define input and output data
X_exp = exp.iloc[:,0:10].values
y_exp = exp.iloc[:,20].values

In [ ]:
# Write results to a text file
with open("output_validation_experiment.txt", "w") as f:
    for name, model in models.items():
        model.fit(X_train,y_train)
        y_predict = model.predict(X_exp)
        report = classification_report(y_exp, y_predict)
        cf = confusion_matrix(y_exp, y_predict)

        f.write(f"Model: {name}\n")
        f.write(f"confusion matrix:\n {cf}\n")
        f.write(f"classification report: \n {report}\n")
        f.write(f"=====================================================\n")
print("Evaluation results exported to .txt file")  